In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

str_tab = 8
minIP = 0
url = 'https://www.fangraphs.com/leaders.aspx?pos=all&stats=pit&lg=all&qual={0}&type={1}&season=2022&month=0&season1=2022&ind=0&team=&rost=&age=&filter=&players=&startdate=&enddate=&page=1_30'.format(minIP, str_tab)
page = requests.get(url)
soup = BeautifulSoup(page.text, 'lxml')



table1 = soup.find('table', id='LeaderBoard1_dg1_ctl00')
headers = []
div_pages = soup.find("div", {"class": "rgWrap rgInfoPart"})
texts = [r.text.strip() for r in div_pages]
pages = int(texts[3])
for i in table1.find_all('th'):
    title = i.text
    headers.append(title)        
mydataComplete = pd.DataFrame(columns = headers)

for page_number in range(0, pages):
    page_number_page = page_number+1
    url = 'https://www.fangraphs.com/leaders.aspx?pos=all&stats=pit&lg=all&qual={0}&type={1}&season=2022&month=0&season1=2022&ind=0&team=&rost=&age=&filter=&players=&startdate=&enddate=&page={2}_30'.format(minIP, str_tab, page_number_page)
    
    # Create object page
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'lxml')
    table1 = soup.find('table', id='LeaderBoard1_dg1_ctl00')
    headers = []
    for i in table1.find_all('th'):
        title = i.text
        headers.append(title)            
    mydata = pd.DataFrame(columns = headers)
    
    idx = 0
    for j in table1.find_all('tr')[1:]:
        row_data = j.find_all('td')
        row = [i.text for i in row_data]
        length = len(mydata)
        if idx<=1:
            idx=idx+1
            continue
        mydata.loc[length] = row
    frames = [mydataComplete, mydata]
    df_pages = pd.concat(frames)
    mydataComplete = df_pages 

mydataComplete['IP'] = mydataComplete['IP'].astype(float)
picherts_type = mydataComplete.nlargest(156, 'IP')
threshold_ip_star = picherts_type['IP'].min()
mydataComplete.loc[mydataComplete['IP']>=threshold_ip_star, 'Type']= 'Starter'
mydataComplete.loc[mydataComplete['IP']<threshold_ip_star, 'Type']= 'Reliever'
picherts_type = mydataComplete[['Name','Team', 'Type']]
picherts_type.to_csv('data/picherts_type.csv', index=False)
